In [33]:
import pandas as pd 
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [23]:
#MODEL

In [3]:
ad = pd.read_csv("Advertising.csv", usecols = [1,2,3,4])
df = ad.copy()
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [7]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict 

In [8]:
#Bağımlı değişkeni silerek sadece bağımsızları bıraktım
X = df.drop("sales", axis = 1)
y = df["sales"]
#80 e 20 oranda bölündü test_size
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)


In [11]:
X_train.shape

(160, 3)

In [12]:
X_test.shape        

(40, 3)

In [13]:
training = df.copy()
training.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [ ]:
## Statsmodels numaradan

In [16]:
lm = sm.OLS(y_train, X_train)
model = lm.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  sales   R-squared (uncentered):                   0.982
Model:                            OLS   Adj. R-squared (uncentered):              0.982
Method:                 Least Squares   F-statistic:                              2935.
Date:                Fri, 23 Oct 2020   Prob (F-statistic):                   1.28e-137
Time:                        15:44:04   Log-Likelihood:                         -336.65
No. Observations:                 160   AIC:                                      679.3
Df Residuals:                     157   BIC:                                      688.5
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TV             0.0531      0.001     36.467      0.000       0.050       0.056
radio          0.2188      0.011     20.138      0.000       0.197       0.240
newspaper      0.0239      0.008      3.011      0.003       0.008       0.040
==============================================================================
Omnibus:                       11.405   Durbin-Watson:                   1.895
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               15.574
Skew:                          -0.432   Prob(JB):                     0.000415
Kurtosis:                       4.261   Cond. No.                         13.5
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
#r kare değerimiz arttı çünkü bağımsız değişkenler arttı

In [19]:
#scikit-learn

In [20]:
lm = LinearRegression()
model = lm.fit(X_train,y_train)

In [21]:
model.intercept_#sabit katsayı (herhangi bir harcama olmasa)

2.979067338122629

In [22]:
model.coef_#diğer birim başına gelir artışları 

array([0.04472952, 0.18919505, 0.00276111])

In [24]:
#TAHMİN

In [25]:
#Sales = 2.9791 + TV*0.0447 + radio*0.1892 + newspaper*0.0028
#çoklu doğrusal regresyon formülü
#30 birim tv 10 birim radyo 40 birim gazete harcaması sonucu tahmini satış değeri ne olur ?

In [26]:
2.9791 + 30*0.0447 + 10*0.1892 + 40*0.0028

6.3241000000000005

In [30]:
yeni_veri = [[30],[10],[40]]#tek doğrusal regresyonda bu şekilde birden fazla tahmin almıştık burda 3 değişkenli olduğundan 3 değişken ifadelerini verdik
yeni_veri = pd.DataFrame(yeni_veri).T
yeni_veri

,0,1,2
0,30,10,40


In [31]:
model.predict(yeni_veri)

array([6.32334798])

In [32]:
#tahmin başarımızı bakalım

In [36]:
rmse = np.sqrt(mean_squared_error(y_train, model.predict(X_train)))#gerçek değerle tahmin değerlerini veririz

In [38]:
rmse#eğitim hatamız 

1.644727765644337

In [39]:
rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
#eğitim veri setini girerek oluşturduğum modeli kullanarak daha önce hiç görmediği ve 
#bağımsız değişkenlerden oluşan değerlerini bulucaz
rmse#test seti hatamız

1.78159966153345

In [44]:
#MODEL TUNİNG

In [118]:
X = df.drop("sales", axis = 1)
y = df["sales"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 144)

lm = LinearRegression()
model = lm.fit(X_train,y_train)

In [119]:
np.sqrt(mean_squared_error(y_train,model.predict(X_train)))

1.6748559274650712

In [120]:
np.sqrt(mean_squared_error(y_test,model.predict(X_test)))

1.6640263686701031

In [121]:
model.score(X_train, y_train)

0.8971614078663419

In [122]:
#random_state değeri değiştikçe ortalama hata karesi karekökü ve score u değişti
#bi kısmını seçeceğiz fakat hangi belirli kısmını seçceğiz problemi var ortadan kaldırmak için
#crossvalidation kullanılır

In [123]:
#Katlı Crossvalidation Yöntemi

In [124]:
#10 katlı crossvalidation 10 farklı r2 değerleri gösterir
cross_val_score(model, X, y,
                cv = 10, scoring = "r2")

array([0.87302696, 0.8581613 , 0.92968723, 0.89013272, 0.93146498,
       0.93138735, 0.7597901 , 0.91217097, 0.83891753, 0.92882311])

In [125]:
cross_val_score(model, X_train, y_train,
                cv = 10, scoring = "r2").mean()#modelimizin doğrulanmış r2 değeridir

0.8733783298422942

In [126]:
#eğitim setimiz için 10 tane eğitim hatası oluşturuldu
np.sqrt(-cross_val_score(model, X_train, y_train,
                         cv = 10, scoring = "neg_mean_squared_error"))

array([1.94167741, 1.17857794, 1.14676306, 1.82238742, 2.97070189,
       1.5424871 , 1.51235118, 1.02817521, 1.73070108, 1.77552332])

In [127]:
#ortalama eğitim seti hata değeri
np.sqrt(-cross_val_score(model, X_train, y_train, cv = 10,
                         scoring = "neg_mean_squared_error")).mean()

1.6649345607872932

In [129]:
#ortalama test hatası (gerçek hatayı validation ile buluruz)
np.sqrt(-cross_val_score(model, X_test, y_test, cv = 10,
                         scoring = "neg_mean_squared_error")).mean()

1.7399924960346649